In [31]:
from torch import nn
import torch
from torchaudio import models

In [10]:
x = torch.rand(4, 80, 320)
x_len = torch.LongTensor([320, 320, 320, 320])

In [4]:
vocab_size = 123

In [32]:
models.Conformer(50, )

TypeError: __init__() missing 5 required positional arguments: 'input_dim', 'num_heads', 'ffn_dim', 'num_layers', and 'depthwise_conv_kernel_size'

In [70]:
def init_sub_block(
    input_dim, model_dim, kernel_size=3, stride=2,
):
    return nn.Sequential(
        nn.Conv1d(input_dim, model_dim, kernel_size=3, stride=2, padding=1),
        nn.ReLU(),
    )

def calc_length(lengths, padding=1, kernel_size=3, stride=2, ceil_mode=False, repeat_num=1):
    add_pad: float = (padding * 2) - kernel_size
    one: float = 1.0
    for i in range(repeat_num):
        lengths = torch.div(lengths.to(dtype=torch.float) + add_pad, stride) + one
        if ceil_mode:
            lengths = torch.ceil(lengths)
        else:
            lengths = torch.floor(lengths)
    return lengths.to(dtype=torch.int)

class EffConformer(nn.Module):
    def __init__(
        self,
        input_dim=80,
        model_dim=144,
        num_heads=4,
        num_layers=[2, 2, 4],
        model_kernel_size=15,
        vocab_size=123
    ):
        super().__init__()
        down = []
        model = []
        self.num_down = len(num_layers)
        for i in range(self.num_down):
            down.append(init_sub_block(input_dim, model_dim))
            model.append(models.Conformer(model_dim, num_heads, model_dim*4,
                                          num_layers[i], model_kernel_size))
            input_dim = model_dim
        self.down = nn.ModuleList(down)
        self.model = nn.ModuleList(model)
        self.out = nn.Linear(model_dim, vocab_size)
        
    def forward(self, x, x_len):
        for i in range(len(self.down)):
            x = self.down[i](x)
            x_len = calc_length(x_len)
            x = x.permute(0, 2, 1)
            x, x_len = self.model[i](x, x_len)
            if i < self.num_down - 1:
                x = x.permute(0, 2, 1)
        o = self.out(x)
        return o, x_len

In [71]:
model = EffConformer()
model, sum(p.numel() for p in model.parameters())

(EffConformer(
   (down): ModuleList(
     (0): Sequential(
       (0): Conv1d(80, 144, kernel_size=(3,), stride=(2,), padding=(1,))
       (1): ReLU()
     )
     (1-2): 2 x Sequential(
       (0): Conv1d(144, 144, kernel_size=(3,), stride=(2,), padding=(1,))
       (1): ReLU()
     )
   )
   (model): ModuleList(
     (0-1): 2 x Conformer(
       (conformer_layers): ModuleList(
         (0-1): 2 x ConformerLayer(
           (ffn1): _FeedForwardModule(
             (sequential): Sequential(
               (0): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
               (1): Linear(in_features=144, out_features=576, bias=True)
               (2): SiLU()
               (3): Dropout(p=0.0, inplace=False)
               (4): Linear(in_features=576, out_features=144, bias=True)
               (5): Dropout(p=0.0, inplace=False)
             )
           )
           (self_attn_layer_norm): LayerNorm((144,), eps=1e-05, elementwise_affine=True)
           (self_attn): MultiheadAttenti

In [72]:
o, o_len = model(x, x_len)
o.size(), o_len

(torch.Size([4, 40, 123]), tensor([40, 40, 40, 40], dtype=torch.int32))